### 1. Libraries

In [1]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                            import Path

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
import cupy as cp


2025-04-30 17:29:35.732585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746048575.806144   24525 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746048575.825875   24525 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746048575.977899   24525 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746048575.977925   24525 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746048575.977926   24525 computation_placer.cc:177] computation placer alr

### 2. Load directory

In [2]:
# Main directory 
mainPath = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5"
#mainPath = "/home/guiomar/Desktop/CODES/DLCODES-VER-5"

directory= os.path.abspath(mainPath)

# Add directory to sys.path
sys.path.append(directory)

# import script where parameters are defined
import param  

### 3. Defining parameters

In [3]:
# Vizualizancing main directories 
list_of_paths = param.list_paths 

# Defining amount of train, test images   
n_split=param.n_split

# Defining a number of samples
n_sample=param.n_sample

# For resizing dimentions
h=param.img_height
w=param.img_width

'''
Showing parameters
'''
for i,j in enumerate(list_of_paths):
    print(f'{i} => {j}')
print(f'\nDimensions resized h = {h}, w = {w}')
print(f'Total data {n_sample}') 

0 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Geometry
1 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Magnitude
2 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Pression
3 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U001
4 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U002

Dimensions resized h = 128, w = 256
Total data 20000


### 4. Process data

In [4]:
#method for reading image    
def get_img(img_name):
    return ski.io.imread(img_name)

# method for turning to a grey or binary image 
def processing(image,option=True):
        x = get_img(image)  
        x = rgb2gray(x)       # It returns a grayscale image with floating point values in the range from 0 to 1
        x =cv.resize(x,(w,h)) # Reshape image 
    
        # Binary option otherwise gray
        if (option):
            x=ski.util.img_as_ubyte(x)  # Convert it back to the original data type and the data range back 0 to 255. 
                                        # It is often better to use image values represented by floating point values 
            best_value_threshold=np.round(filters.threshold_otsu(x)) #  Otsu’s method calculates an “optimal” threshold

            _,x= cv.threshold(x, best_value_threshold, 255, cv.THRESH_BINARY)
            x=x/255.
        x=x[:, :, np.newaxis]
        return x

In [5]:
# Method for processing by batches 
def  process_by_batches(list_of_paths,number_batch,name_file,option=True):
    files=sorted([os.path.join(list_of_paths,fname) for fname in os.listdir(list_of_paths) if fname.endswith(".png")])
    files=files[1:]
    n_files=len(files)
    new_file= param.save_in + '/' + name_file
    os.makedirs(new_file, exist_ok=True)
    for k,start in enumerate(range(0,n_files,number_batch)):
         batch=[]
         end = min(start + number_batch, n_files)
         file_batches=files[start:end]
         for img in file_batches:
             x = processing(img,option)

             # x = x * geo
             
             batch.append(x)  
         print(k,'*************************')   
         image_processed = new_file + f'/batch_images_{k:03d}.npy'
         np.save(image_processed,batch,allow_pickle=True)   
         del batch
         gc.collect()
        
# Method for concatenating all batches
def concatenate_batches(name_file):
    file= param.save_in + '/' + name_file
    path=Path(file)
    files_npy=sorted(list(path.glob('*.npy')))
    batches = []
    for f in files_npy:
        array = np.load(f, allow_pickle=True, mmap_mode='r') 
        batches.append(array)
    result = np.concatenate(batches, axis=0)    
    del batches
    return result
    
# Method for split data    
def split_data(result):
    n_total = result.shape[0]  
    n_train = int(n_split * n_total)
    test = result[:n_train]
    train = result[n_train:]
    return train,test
    
#*********************************************************************************   

# Create a mask
def create_mask(u,v):
    m,n,j,i = u.shape
    w = np.zeros((m, n, j, i), dtype=np.float32)
    for k in range(m):
         w[k] = u[k] * v[k]
    return w


In [6]:
# Call method for processing by batches
# It creates 40 batches of 500 arrays each one 

'''
process_by_batches(list_of_paths[0],param.n_batch,'Geo')
process_by_batches(list_of_paths[1],param.n_batch,'Mag',False)
process_by_batches(list_of_paths[2],param.n_batch,'P',False)
process_by_batches(list_of_paths[3],param.n_batch,'Vx',False)
process_by_batches(list_of_paths[4],param.n_batch,'Vy',False)
'''

"\nprocess_by_batches(list_of_paths[0],param.n_batch,'Geo')\nprocess_by_batches(list_of_paths[1],param.n_batch,'Mag',False)\nprocess_by_batches(list_of_paths[2],param.n_batch,'P',False)\nprocess_by_batches(list_of_paths[3],param.n_batch,'Vx',False)\nprocess_by_batches(list_of_paths[4],param.n_batch,'Vy',False)\n"

In [7]:
# Call method  for concatenating all 40 batches in one array of 20000 samples

geo = concatenate_batches('Geo')
mag = concatenate_batches('Mag')
P =  concatenate_batches('P')
Vx = concatenate_batches('Vx')
Vy = concatenate_batches('Vy')

In [9]:
geo.shape

(20000, 128, 256, 1)

In [10]:
# Splitting data in 80 % for training and 20 % for testing
geo_train, geo_test = split_data(geo)
mag_train, mag_test = split_data(mag)
p_train, p_test = split_data(P)
vx_train, vx_test = split_data(Vx)
vy_train, vy_test = split_data(Vy)

In [11]:
# Showing the shape of training data and testing
print(f'{geo_train.shape} and {geo_test.shape}')

(16000, 128, 256, 1) and (4000, 128, 256, 1)


In [12]:
# Masking

'''
mag_masked_train = create_mask(geo_train,mag_train)
mag_masked_test  = create_mask(geo_test,mag_test)
vy_masked_test =  create_mask(geo_test,vy_test)
p_masked_train = create_mask(geo_train,p_train)
p_masked_test =  create_mask(geo_test,p_test)
vx_masked_train = create_mask(geo_train,vx_train)
vx_masked_test =  create_mask(geo_test,vx_test)
vy_masked_train = create_mask(geo_train,vy_train)
'''

'\nmag_masked_train = create_mask(geo_train,mag_train)\nmag_masked_test  = create_mask(geo_test,mag_test)\nvy_masked_test =  create_mask(geo_test,vy_test)\np_masked_train = create_mask(geo_train,p_train)\np_masked_test =  create_mask(geo_test,p_test)\nvx_masked_train = create_mask(geo_train,vx_train)\nvx_masked_test =  create_mask(geo_test,vx_test)\nvy_masked_train = create_mask(geo_train,vy_train)\n'

In [13]:
(m, n, j, i)=geo_train.shape
mag_masked_train = np.zeros((m, n, j, i), dtype=np.float32)
p_masked_train = np.zeros((m, n, j, i), dtype=np.float32)
vx_masked_train = np.zeros((m, n, j, i), dtype=np.float32)
vy_masked_train = np.zeros((m, n, j, i), dtype=np.float32)

In [14]:
mag_masked_train.shape

(16000, 128, 256, 1)

In [15]:
geo_train.shape

(16000, 128, 256, 1)

In [16]:
vy_train.shape

(16000, 128, 256, 1)

In [17]:
for k in range(m):
    mag_masked_train[k] = geo_train[k] * mag_train[k]
del mag_train

In [18]:
for k in range(m):
    p_masked_train[k] = geo_train[k] * p_train[k]
del p_train

In [19]:
for k in range(m):
    vx_masked_train[k] = geo_train[k] * vx_train[k]
del vx_train

In [ ]:
for k in range(m):
    vy_masked_train[k] = geo_train[k] * vy_train[k]
del vy_train

In [ ]:
# Create a tensor of four channels
data_merged_train = np.concatenate((mag_masked_train, p_masked_train, vx_masked_train, vy_masked_train), axis=-1)
data_merged_tests = np.concatenate((mag_masked_test,p_masked_test,vx_masked_test, vy_masked_test), axis=-1)

# print dataset values
print(data_merged_train.shape)
print(data_merged_tests.shape)

#### 4.3 Visualizing dataset 

In [ ]:
def show_image(img1,img2,img3,img4,title=None,opt=None,prediction=None):
    if opt==None and prediction==None:
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 5), sharey=True)
        axes[0, 0].set_xticks([]);axes[0, 0].set_yticks([]);
        axes[1, 0].set_xticks([]); axes[1, 0].set_yticks([]);
        axes[0, 1].set_xticks([]); axes[0, 1].set_yticks([]);
        axes[1, 1].set_xticks([]); axes[1, 1].set_yticks([]);
        
        axes[0, 0].imshow(img1,origin='upper',cmap='gray')
        axes[0, 0].set_title('Input',fontsize=14)
        axes[0, 0].set_ylabel('Geometry',fontsize=14)

        axes[0, 1].imshow(img2,origin='upper',cmap='gray')
        axes[0, 1].set_title('Output-Magnitude',fontsize=14)
        axes[0, 1].set_ylabel('V',fontsize=14)

        axes[1, 0].imshow(img3,origin='upper',cmap='gray')
        axes[1, 0].set_title('Output x-velocity',fontsize=14)
        axes[1, 0].set_ylabel('Vx',fontsize=14)

        axes[1, 1].imshow(img4,origin='upper',cmap='gray')
        axes[1, 1].set_title('Output-Pression',fontsize=14)
        axes[1, 1].set_ylabel('P',fontsize=14)

        if title is not None: fig.suptitle(title)
    
    else:
        fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(16, 5), sharey=True)
        ax1.set_xticks([]); ax1.set_yticks([]);
        ax2.set_xticks([]); ax2.set_yticks([]);
        ax1.imshow(a1,origin='upper',cmap='gray')
        ax1.set_title('Real',fontsize=14)
        #ax1.set_xlabel('Geometry')
        ax1.set_ylabel('u',fontsize=14)
        ax2.imshow(a2,origin='upper',cmap='gray')
        ax2.set_title('Predicted',fontsize=14)
        #ax2.set_xlabel('u')
        ax2.set_ylabel(prediction,fontsize=16)
        if title is not None: fig.suptitle(title)
        

In [ ]:
img_random=random.sample(range(1, geo_train.shape[0]), 1)
show_image(geo_train[img_random[0]],mag_train[img_random[0]],
           vx_train[img_random[0]],p_train[img_random[0]],
           title=f"Training data sample N°{img_random[0]}")

In [ ]:
img_random=random.sample(range(1, geo_train.shape[0]), 1)
show_image(geo_train[img_random[0]],mag_masked_train[img_random[0]],
           vx_masked_train[img_random[0]],p_masked_train[img_random[0]],
           title=f"Training data sample N°{img_random[0]}")

### 5. Hiperparameters

In [ ]:
# Print hiperparameters
num_classes =param.num_classes
num_epochs =param.num_epochs
batch_size=param.batch_size
patience=param.patience
LR=param.LR
print(f'Classes= {num_classes},\n Epochs = {num_epochs},\n batches = {batch_size},\n lr = {LR}')

### 6. Architecture

In [ ]:
# model

image_input = Input((h, w, param.channel))

# 64 3x3 convolutions followed by a batchnormalization and max pool
conv1 = Conv2D(64, (3, 3), padding='same')(image_input)
conv1= BatchNormalization()(conv1)
conv1 = Activation("relu")(conv1)
conv1 = Conv2D(64, (3, 3), padding='same')(conv1)
conv1= BatchNormalization()(conv1)
conv1 = Activation("relu")(conv1)
pool1=MaxPooling2D((2,2))(conv1)

# 128 3x3 convolutions followed by a batchnormalization and max pool
conv2 = Conv2D(128, (3, 3), padding='same')(pool1)
conv2= BatchNormalization()(conv2)
conv2 = Activation("relu")(conv2)
conv2 = Conv2D(128, (3, 3), padding='same')(conv2)
conv2= BatchNormalization()(conv2)
conv2 = Activation("relu")(conv2)
pool2=MaxPooling2D((2,2))(conv1)

# 256 3x3 convolutions followed by a batchnormalization and max pool
conv3 = Conv2D(256, (3, 3), padding='same')(pool2)
conv3= BatchNormalization()(conv3)
conv3 = Activation("relu")(conv3)
conv3 = Conv2D(256, (3, 3), padding='same')(conv3)
conv3= BatchNormalization()(conv3)
conv3 = Activation("relu")(conv3)
pool3=MaxPooling2D((2,2))(conv3)

# 512 3x3 convolutions followed by a batchnormalization and max pool
conv4 = Conv2D(512, (3, 3), padding='same')(pool3)
conv4= BatchNormalization()(conv4)
conv4 = Activation("relu")(conv4)
conv4 = Conv2D(512, (3, 3), padding='same')(conv4)
conv4= BatchNormalization()(conv4)
conv4= Activation("relu")(conv4)
pool4=MaxPooling2D((2,2))(conv4)

# 1024 Bottleneck  3x3 convolutions
conv5 = Conv2D(1024, (3,3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv5)

# 512 3x3 transpose convolution and concate
conv6=Conv2DTranspose(512,(2,2),strides=(2, 2),padding="same")(conv5) # remark: if it's used padding=valid, apply copy and crop the skip features
up6=concatenate([conv6,conv4])
conv6=Conv2D(512, (3, 3), activation="relu", padding="same")(up6)
conv6=Conv2D(512, (3, 3), activation="relu", padding="same")(conv6)

# 216 3x3 transpose convolution and concate
conv7=Conv2DTranspose(216,(2,2),strides=(2, 2),padding="same")(conv6) 
up7=concatenate([conv7,conv3])
conv7=Conv2D(216, (3, 3), activation="relu", padding="same")(up7)
conv7=Conv2D(216, (3, 3), activation="relu", padding="same")(conv7)

# 128 3x3 transpose convolution and concate
conv8=Conv2DTranspose(128,(2,2),strides=(2, 2),padding="same")(conv7) 
up8=concatenate([conv8,conv3])
conv8=Conv2D(128, (3, 3), activation="relu", padding="same")(up8)
conv8=Conv2D(128, (3, 3), activation="relu", padding="same")(conv8)

# 64 3x3 transpose convolution and concate
conv9=Conv2DTranspose(64,(2,2),strides=(2, 2),padding="same")(conv8) 
up9=concatenate([conv9,conv2])
conv9=Conv2D(64, (3, 3), activation="relu", padding="same")(up9)
conv9=Conv2D(64, (3, 3), activation="relu", padding="same")(conv9)

# final 1x1 convolutions to get to the correct depth dim (num_classes=4 for v, v-x, v-y, p)
output = Conv2D(num_classes, (1, 1), activation="relu", padding="same")(conv9)

output = image_input * output

# Contrut  model
model = models.Model(image_input, output, name="U-Net")

model.summary()


In [ ]:
# Early stops the training if validation loss doesn't improve after a given patience
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

# compile the model with loss and optimizer
optimizer = tf.keras.optimizers.RMSprop(LR) 
model.compile(optimizer=optimizer,
              loss = tf.keras.losses.MeanSquaredError(),
              metrics =[tf.keras.metrics.MeanAbsoluteError()])

# Plot the resulting model architecture
tf.keras.utils.plot_model(model, to_file="//home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Result_01/U_Net.png")
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Result_01/Best_weights.weights.h5",save_weights_only=True)

# Train model
model.fit(geo_train,data_merged_train,
          batch_size=batch_size,
    
)



In [ ]:
model_fitted=model.fit(self.train_data,self.train_data_sol,
                                   batch_size=self.batch_size,
                                   validation_split = 0.2,
                                   # validation_data=(self.val_data, self.val_data_sol),
                                    epochs=self.epoch,callbacks=[self.early_stopping(),checkpoint_cb])


# train model
model.fit(train_geometries, train_steady_flows,
          batch_size=batch_size,
          epochs=num_epochs,
          verbose=1,
          validation_data=(test_geometries, test_steady_flows))

In [ ]:
       
'''
def split_data(name_file):
    file= param.save_in + '/' + name_file
    path=Path(file)
    files_npy=list(path.glob('*.npy'))
    list_batchs = [np.load(f) for f in files_npy]
    images_arrays = np.concatenate(list_batchs, axis=0) 
    x_train, x_test = train_test_split(images_arrays,test_size=n_split, shuffle=False)
    return x_train, x_test
'''
'''
(m, n, j, i)=geo_train.shape
w = np.zeros((m, n, j, i), dtype=np.float32)
for k in range(m):
    w[k] = geo_train[k] * mag_train[k]
'''